# Herbarium 2021 FGVC8 - Plan

Goal: Build a GPU-accelerated image classification pipeline to achieve a medal-worthy macro-F1 on the leaderboard.

Plan:
- Env check: verify GPU availability and correct CUDA stack.
- Data audit: load train/test metadata.json, inspect fields, class counts, image-path mapping, and sample_submission format.
- CV protocol: stratified K-fold by label; ensure deterministic seeds.
- Baseline model: torchvision pretrained backbone (e.g., convnext_tiny / efficientnet_v2_s) fine-tune with mixed precision.
- Augmentations: standard image aug (resize, crop, color jitter, flips); class-balanced sampling.
- Training loop: early stopping, lr scheduling (OneCycle/Step), label smoothing; log progress by epoch/fold.
- Inference: TTA, save predictions to submission.csv in required format.
- Iteration: cache OOF and test logits; analyze errors; consider higher res or stronger backbones; blend if time.

Checkpoints requiring expert review:
- After this plan and environment/data audit
- After baseline CV setup
- After first baseline training results
- After improvements/ensembling decisions

Notes:
- Keep jobs fast; subsample for smoke tests first.
- Always print timing per fold and verify artifacts exist.

In [1]:
import os, json, sys, subprocess, time, textwrap, pandas as pd
from pathlib import Path

def run(cmd):
    print('> ', ' '.join(cmd), flush=True)
    return subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True).stdout

print('Env check: nvidia-smi')
print(run(['bash','-lc','nvidia-smi || true']))
print('Python:', sys.version)

base = Path('.')
train_dir = base/'train'
test_dir = base/'test'
ss_path = base/'sample_submission.csv'

print('List dir train/test:')
print('train exists:', train_dir.exists(), 'test exists:', test_dir.exists())
print('train/images dirs:', len(list((train_dir/'images').glob('*'))))
print('test/images dirs:', len(list((test_dir/'images').glob('*'))))

def load_json(p):
    with open(p, 'r') as f:
        return json.load(f)

train_meta = load_json(train_dir/'metadata.json')
test_meta = load_json(test_dir/'metadata.json')
print('train_meta keys:', list(train_meta.keys()))
print('test_meta keys:', list(test_meta.keys()))

# Inspect a few entries
def head_dict(d, n=3):
    # if dict of lists or dict with key 'images', try to summarize
    if isinstance(d, dict) and 'images' in d:
        imgs = d['images'][:n]
        print('images sample:', imgs)
    elif isinstance(d, dict) and 'annotations' in d:
        print('annotations sample:', d['annotations'][:n])
    else:
        # print first n key:val pairs
        for i, (k,v) in enumerate(d.items()):
            if i>=n: break
            print(k, type(v), (v if isinstance(v,(int,str,float)) else '...'))

print('Train meta head:')
head_dict(train_meta)
print('Test meta head:')
head_dict(test_meta)

# Try to infer schema commonly used in Herbarium competitions
# Expect fields like: images (list of dicts with file_name, id), annotations (list with image_id, category_id), categories (list with id, name)
images = train_meta.get('images', [])
ann = train_meta.get('annotations', [])
cats = train_meta.get('categories', [])
print(f'Counts - images: {len(images)}, annotations: {len(ann)}, categories: {len(cats)}')

if images and ann and cats:
    import pandas as pd
    df_img = pd.DataFrame(images)
    df_anno = pd.DataFrame(ann)
    df_cat = pd.DataFrame(cats)
    print('df_img columns:', df_img.columns.tolist())
    print('df_anno columns:', df_anno.columns.tolist())
    print('df_cat columns:', df_cat.columns.tolist())
    # Merge labels
    if 'id' in df_img.columns and 'image_id' in df_anno.columns:
        df = df_anno.merge(df_img, left_on='image_id', right_on='id', how='left')
        if 'category_id' in df.columns:
            print('Train rows:', len(df))
            print('Unique classes:', df['category_id'].nunique())
            print('Label distribution (head):')
            print(df['category_id'].value_counts().head(10))
            # Build relative image path
            # Expect file_name in images and path under train/images/
            if 'file_name' in df_img.columns:
                pass

if ss_path.exists():
    ss = pd.read_csv(ss_path)
    print('sample_submission head:')
    print(ss.head())
else:
    print('sample_submission.csv missing')

print('Done audit at', time.strftime('%Y-%m-%d %H:%M:%S'))

Env check: nvidia-smi
>  bash -lc nvidia-smi || true


Sat Sep 27 04:39:42 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.144.06             Driver Version: 550.144.06     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A10-24Q                 On  |   00000002:00:00.0 Off |                    0 |
| N/A   N/A    P0             N/A /  N/A  |     182MiB /  24512MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

train_meta keys: ['annotations', 'categories', 'images', 'info', 'institutions', 'licenses']
test_meta keys: ['images', 'info', 'licenses']
Train meta head:
images sample: [{'file_name': 'images/604/92/1608432.jpg', 'height': 1000, 'id': 1608432, 'license': 0, 'width': 671}, {'file_name': 'images/604/92/796948.jpg', 'height': 1000, 'id': 796948, 'license': 0, 'width': 678}, {'file_name': 'images/604/92/994447.jpg', 'height': 1000, 'id': 994447, 'license': 0, 'width': 671}]
Test meta head:
images sample: [{'file_name': 'images/000/0.jpg', 'height': 1000, 'id': '0', 'license': 0, 'width': 666}, {'file_name': 'images/000/1.jpg', 'height': 1000, 'id': '1', 'license': 0, 'width': 672}, {'file_name': 'images/000/2.jpg', 'height': 1000, 'id': '2', 'license': 0, 'width': 669}]
Counts - images: 1779953, annotations: 1779953, categories: 64500


df_img columns: ['file_name', 'height', 'id', 'license', 'width']
df_anno columns: ['category_id', 'id', 'image_id', 'institution_id']
df_cat columns: ['family', 'id', 'name', 'order']


Train rows: 1779953
Unique classes: 64500
Label distribution (head):
category_id
42811    2647
25229    1713
48372    1630
42843    1324
22344    1260
42580    1152
11117    1091
56357    1058
26119    1007
58922     987
Name: count, dtype: int64
sample_submission head:
   Id  Predicted
0   0          0
1   1          0
2   2          0
3   3          0
4   4          0
Done audit at 2025-09-27 04:39:49


In [2]:
# Safety-net baseline: Global majority-class submission
import json, pandas as pd
from pathlib import Path

base = Path('.')
train_meta = json.load(open(base/'train'/'metadata.json','r'))
test_meta = json.load(open(base/'test'/'metadata.json','r'))

df_img = pd.DataFrame(train_meta['images'])
df_anno = pd.DataFrame(train_meta['annotations'])
df = df_anno.merge(df_img, left_on='image_id', right_on='id', how='left')
mode_label = int(df['category_id'].mode().iloc[0])
print('Global mode category_id:', mode_label)

ss = pd.read_csv(base/'sample_submission.csv')
print('Sample submission shape:', ss.shape)
sub = ss.copy()
sub['Predicted'] = mode_label
sub.to_csv('submission.csv', index=False)
print('Wrote submission.csv with global majority class. Head:')
print(sub.head())

Global mode category_id: 42811
Sample submission shape: (477806, 2)
Wrote submission.csv with global majority class. Head:
   Id  Predicted
0   0      42811
1   1      42811
2   2      42811
3   3      42811
4   4      42811


In [3]:
# Metadata-only F0-conditioned baseline
import json, pandas as pd, numpy as np
from pathlib import Path

base = Path('.')
train_meta = json.load(open(base/'train'/'metadata.json','r'))
test_meta = json.load(open(base/'test'/'metadata.json','r'))
ss = pd.read_csv(base/'sample_submission.csv')

df_img = pd.DataFrame(train_meta['images'])[['file_name','width','height','id']]
df_anno = pd.DataFrame(train_meta['annotations'])[['image_id','category_id']]
df_tr = df_anno.merge(df_img, left_on='image_id', right_on='id', how='left')

def parse_F0_F1(s):
    # s like 'images/604/92/1608432.jpg'
    parts = s.split('/')
    f0 = parts[1] if len(parts) > 1 else ''
    f1 = parts[2] if len(parts) > 2 else ''
    return f0, f1

F0_F1 = df_tr['file_name'].map(parse_F0_F1)
df_tr['F0'] = [t[0] for t in F0_F1]
df_tr['F1'] = [t[1] for t in F0_F1]
df_tr['aspect'] = (df_tr['width'] / df_tr['height']).astype(float)
width_bins = [0, 600, 700, 800, 900, 1000, 1100, 2000]
height_bins = [0, 600, 700, 800, 900, 1000, 1100, 2000]
df_tr['wb'] = pd.cut(df_tr['width'], bins=width_bins, include_lowest=True).astype(str)
df_tr['hb'] = pd.cut(df_tr['height'], bins=height_bins, include_lowest=True).astype(str)

global_mode = int(df_tr['category_id'].mode().iloc[0])
print('Global mode:', global_mode)

def mode_map(df, keys):
    # returns dict mapping tuple(keys) -> modal category_id
    grp = df.groupby(keys)['category_id'].agg(lambda x: x.value_counts().idxmax())
    return grp.to_dict()

m_F0_wb_hb = mode_map(df_tr, ['F0','wb','hb'])
m_F0_wb = mode_map(df_tr, ['F0','wb'])
m_F0_hb = mode_map(df_tr, ['F0','hb'])
m_F0 = mode_map(df_tr, ['F0'])

# Build test features
df_te = pd.DataFrame(test_meta['images'])[['id','file_name','width','height']].copy()
F0_F1_te = df_te['file_name'].map(parse_F0_F1)
df_te['F0'] = [t[0] for t in F0_F1_te]
df_te['F1'] = [t[1] for t in F0_F1_te]
df_te['wb'] = pd.cut(df_te['width'], bins=width_bins, include_lowest=True).astype(str)
df_te['hb'] = pd.cut(df_te['height'], bins=height_bins, include_lowest=True).astype(str)

def predict_row(r):
    k3 = (r['F0'], r['wb'], r['hb'])
    if k3 in m_F0_wb_hb: return int(m_F0_wb_hb[k3])
    k2 = (r['F0'], r['wb'])
    if k2 in m_F0_wb: return int(m_F0_wb[k2])
    k2b = (r['F0'], r['hb'])
    if k2b in m_F0_hb: return int(m_F0_hb[k2b])
    k1 = (r['F0'],)
    if k1 in m_F0: return int(m_F0[r['F0']])
    return global_mode

t0 = pd.Timestamp.now()
preds = df_te.apply(predict_row, axis=1)
elapsed = (pd.Timestamp.now() - t0).total_seconds()
print(f'Predicted {len(preds)} rows in {elapsed:.2f}s')

# Align to sample submission order
sub = ss.copy()
id2pred = dict(zip(df_te['id'].astype(str), preds))
sub['Predicted'] = sub['Id'].astype(str).map(id2pred).fillna(global_mode).astype(int)
sub.to_csv('submission.csv', index=False)
sub.to_csv('submission_f0.csv', index=False)
print('Wrote submission.csv and submission_f0.csv. Head:')
print(sub.head())
print('Coverage check:')
covered = (sub['Predicted'] != global_mode).mean()
print('Fraction not falling back to global:', f'{covered:.3f}')

Global mode: 42811


Predicted 477806 rows in 2.47s


Wrote submission.csv and submission_f0.csv. Head:
   Id  Predicted
0   0         23
1   1         23
2   2         23
3   3         23
4   4         23
Coverage check:
Fraction not falling back to global: 0.998


In [4]:
# Enhanced metadata baseline: add F1 and aspect bins to fallback chain
import json, pandas as pd, numpy as np
from pathlib import Path

base = Path('.')
train_meta = json.load(open(base/'train'/'metadata.json','r'))
test_meta = json.load(open(base/'test'/'metadata.json','r'))
ss = pd.read_csv(base/'sample_submission.csv')

df_img = pd.DataFrame(train_meta['images'])[['file_name','width','height','id']]
df_anno = pd.DataFrame(train_meta['annotations'])[['image_id','category_id']]
df_tr = df_anno.merge(df_img, left_on='image_id', right_on='id', how='left')

def parse_F0_F1(s):
    parts = s.split('/')
    f0 = parts[1] if len(parts) > 1 else ''
    f1 = parts[2] if len(parts) > 2 else ''
    return f0, f1

F0_F1 = df_tr['file_name'].map(parse_F0_F1)
df_tr['F0'] = [t[0] for t in F0_F1]
df_tr['F1'] = [t[1] for t in F0_F1]
df_tr['aspect'] = (df_tr['width'] / df_tr['height']).astype(float)
width_bins = [0, 600, 700, 800, 900, 1000, 1100, 2000]
height_bins = [0, 600, 700, 800, 900, 1000, 1100, 2000]
aspect_bins = [0, 0.6, 0.75, 0.9, 1.0, 1.1, 1.3, 2.5]
df_tr['wb'] = pd.cut(df_tr['width'], bins=width_bins, include_lowest=True).astype(str)
df_tr['hb'] = pd.cut(df_tr['height'], bins=height_bins, include_lowest=True).astype(str)
df_tr['ab'] = pd.cut(df_tr['aspect'], bins=aspect_bins, include_lowest=True).astype(str)

global_mode = int(df_tr['category_id'].mode().iloc[0])
print('Global mode:', global_mode)

def mode_map(df, keys):
    grp = df.groupby(keys)['category_id'].agg(lambda x: x.value_counts().idxmax())
    return grp.to_dict()

# Build maps with increasing specificity
m_F0_F1_wb_hb = mode_map(df_tr, ['F0','F1','wb','hb'])
m_F0_F1_ab   = mode_map(df_tr, ['F0','F1','ab'])
m_F0_wb_hb   = mode_map(df_tr, ['F0','wb','hb'])
m_F0_ab      = mode_map(df_tr, ['F0','ab'])
m_F0_F1      = mode_map(df_tr, ['F0','F1'])
m_F0_wb      = mode_map(df_tr, ['F0','wb'])
m_F0_hb      = mode_map(df_tr, ['F0','hb'])
m_F0         = mode_map(df_tr, ['F0'])

# Test features
df_te = pd.DataFrame(test_meta['images'])[['id','file_name','width','height']].copy()
F0_F1_te = df_te['file_name'].map(parse_F0_F1)
df_te['F0'] = [t[0] for t in F0_F1_te]
df_te['F1'] = [t[1] for t in F0_F1_te]
df_te['aspect'] = (df_te['width'] / df_te['height']).astype(float)
df_te['wb'] = pd.cut(df_te['width'], bins=width_bins, include_lowest=True).astype(str)
df_te['hb'] = pd.cut(df_te['height'], bins=height_bins, include_lowest=True).astype(str)
df_te['ab'] = pd.cut(df_te['aspect'], bins=aspect_bins, include_lowest=True).astype(str)

levels = [
    ('F0_F1_wb_hb', lambda r: m_F0_F1_wb_hb.get((r['F0'], r['F1'], r['wb'], r['hb']))),
    ('F0_F1_ab',    lambda r: m_F0_F1_ab.get((r['F0'], r['F1'], r['ab']))),
    ('F0_wb_hb',    lambda r: m_F0_wb_hb.get((r['F0'], r['wb'], r['hb']))),
    ('F0_ab',       lambda r: m_F0_ab.get((r['F0'], r['ab']))),
    ('F0_F1',       lambda r: m_F0_F1.get((r['F0'], r['F1']))),
    ('F0_wb',       lambda r: m_F0_wb.get((r['F0'], r['wb']))),
    ('F0_hb',       lambda r: m_F0_hb.get((r['F0'], r['hb']))),
    ('F0',          lambda r: m_F0.get((r['F0'],)))
]

pred = np.full(len(df_te), global_mode, dtype=np.int64)
covered = np.zeros(len(df_te), dtype=bool)

for name, fn in levels:
    if covered.all():
        break
    idx = np.where(~covered)[0]
    # vectorized-ish apply on remaining subset
    vals = [fn(df_te.iloc[i]) for i in idx]
    take = [v is not None for v in vals]
    if any(take):
        pred_idx = np.array(idx)[np.array(take)]
        pred_vals = np.array([int(v) for v in np.array(vals, dtype=object)[np.array(take)]])
        pred[pred_idx] = pred_vals
        covered[pred_idx] = True
    print(f'Level {name}: newly covered {covered.mean():.3f} cumul.')

sub = ss.copy()
id2pred = dict(zip(df_te['id'].astype(str), pred.tolist()))
sub['Predicted'] = sub['Id'].astype(str).map(id2pred).fillna(global_mode).astype(int)
sub.to_csv('submission.csv', index=False)
sub.to_csv('submission_f0_ext.csv', index=False)
print('Wrote submission.csv and submission_f0_ext.csv. Head:')
print(sub.head())

Global mode: 42811


Level F0_F1_wb_hb: newly covered 0.000 cumul.


Level F0_F1_ab: newly covered 0.000 cumul.


Level F0_wb_hb: newly covered 0.994 cumul.
Level F0_ab: newly covered 0.994 cumul.
Level F0_F1: newly covered 0.994 cumul.


Level F0_wb: newly covered 0.994 cumul.
Level F0_hb: newly covered 1.000 cumul.
Level F0: newly covered 1.000 cumul.


Wrote submission.csv and submission_f0_ext.csv. Head:
   Id  Predicted
0   0         23
1   1         23
2   2         23
3   3         23
4   4         23


In [5]:
# Install Torch cu121 stack and vision deps; verify GPU
import os, sys, subprocess, shutil
from pathlib import Path

def pip(*args):
    print('> pip', ' '.join(args), flush=True)
    subprocess.run([sys.executable, '-m', 'pip', *args], check=True)

for pkg in ('torch','torchvision','torchaudio'):
    subprocess.run([sys.executable, '-m', 'pip', 'uninstall', '-y', pkg], check=False)

for d in (
    '/app/.pip-target/torch',
    '/app/.pip-target/torchvision',
    '/app/.pip-target/torchaudio',
    '/app/.pip-target/torch-2.4.1.dist-info',
    '/app/.pip-target/torchvision-0.19.1.dist-info',
    '/app/.pip-target/torchaudio-2.4.1.dist-info',
):
    if os.path.exists(d):
        print('Removing', d)
        shutil.rmtree(d, ignore_errors=True)

pip('install', '--no-cache-dir', '--index-url', 'https://download.pytorch.org/whl/cu121', 'torch==2.4.1', 'torchvision==0.19.1', 'torchaudio==2.4.1')
Path('constraints.txt').write_text('torch==2.4.1\ntorchvision==0.19.1\ntorchaudio==2.4.1\n')
pip('install', '-c', 'constraints.txt', 'timm==1.0.9', 'albumentations==1.4.10', 'opencv-python-headless==4.10.0.84', '--upgrade-strategy', 'only-if-needed')

import torch, torchvision
print('torch:', torch.__version__, 'cuda build:', getattr(torch.version,'cuda', None), 'CUDA available:', torch.cuda.is_available())
if torch.cuda.is_available():
    print('GPU:', torch.cuda.get_device_name(0))
else:
    raise SystemExit('CUDA not available; cannot proceed to CNN training')

> pip install --no-cache-dir --index-url https://download.pytorch.org/whl/cu121 torch==2.4.1 torchvision==0.19.1 torchaudio==2.4.1


Looking in indexes: https://download.pytorch.org/whl/cu121


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 799.0/799.0 MB 403.7 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 446.7 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 89.8 MB/s eta 0:00:00


...

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 221.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 206.1 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.7/39.7 MB 148.4 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 KB 438.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.8/129.8 KB 485.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.2/161.2 KB 472.4 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 KB 162.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 KB 486.7 MB/s eta 0:00:00


torch: 2.4.1+cu121 cuda build: 12.1 CUDA available: True
GPU: NVIDIA A10-24Q


In [7]:
# Dummy-ready CNN pipeline scaffold (hot-swap to real images when available)
import os, json, math, random, time, gc
from pathlib import Path
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import torchvision.transforms as T
from PIL import Image
import timm

base = Path('.')
train_dir = base/'train'
test_dir = base/'test'

# Build train/test DataFrames
train_meta = json.load(open(train_dir/'metadata.json','r'))
test_meta = json.load(open(test_dir/'metadata.json','r'))
df_img = pd.DataFrame(train_meta['images'])[['id','file_name','width','height']]
df_anno = pd.DataFrame(train_meta['annotations'])[['image_id','category_id']]
df_cat = pd.DataFrame(train_meta['categories'])[['id','name','family','order']]
df_train = df_anno.merge(df_img, left_on='image_id', right_on='id', how='left')
df_train['path'] = df_train['file_name'].apply(lambda s: str(train_dir / s))
df_test = pd.DataFrame(test_meta['images'])[['id','file_name','width','height']].copy()
df_test['path'] = df_test['file_name'].apply(lambda s: str(test_dir / s))

# Label encoding
unique_cids = sorted(df_train['category_id'].unique())
cid2idx = {c:i for i,c in enumerate(unique_cids)}
idx2cid = np.array(unique_cids, dtype=np.int64)
df_train['label'] = df_train['category_id'].map(cid2idx).astype(np.int64)
num_classes = len(unique_cids)
print('Train rows:', len(df_train), 'Num classes:', num_classes)

def count_jpgs():
    # quick diagnostics for image availability
    import subprocess
    cmd = "bash -lc 'shopt -s nullglob; arr=(train/images/*/*/*.jpg); echo ${#arr[@]}; arr=(test/images/*/*.jpg); echo ${#arr[@]}'"
    out = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True).stdout.strip().splitlines()
    if len(out)>=2:
        print(f'jpg counts \u2192 train: {out[0]}, test: {out[1]}')
    else:
        print('jpg count check unavailable')
count_jpgs()

class HerbariumDataset(Dataset):
    def __init__(self, df, mode='train', img_size=384):
        self.df = df.reset_index(drop=True)
        self.mode = mode
        self.img_size = img_size
        self.tf_train = T.Compose([
            T.RandomResizedCrop(img_size, scale=(0.7,1.0), ratio=(0.75,1.33)),
            T.RandomHorizontalFlip(p=0.5),
            T.ToTensor(),
            T.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225)),
        ])
        self.tf_test = T.Compose([
            T.Resize(img_size, interpolation=T.InterpolationMode.BICUBIC),
            T.CenterCrop(img_size),
            T.ToTensor(),
            T.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225)),
        ])
    def __len__(self):
        return len(self.df)
    def _load_image(self, path):
        try:
            if os.path.exists(path):
                with Image.open(path) as im:
                    return im.convert('RGB')
        except Exception as e:
            pass
        # Fallback dummy if file missing/unreadable
        return Image.new('RGB', (self.img_size, self.img_size), (0,0,0))
    def __getitem__(self, idx):
        r = self.df.iloc[idx]
        img = self._load_image(r['path'])
        if self.mode == 'train':
            img = self.tf_train(img)
            label = int(r['label'])
            return img, label
        else:
            img = self.tf_test(img)
            return img, str(r['id'])

def make_model(backbone='convnext_tiny', num_classes=1):
        model = timm.create_model(backbone, pretrained=True, num_classes=num_classes)
        model = model.to('cuda' if torch.cuda.is_available() else 'cpu')
        return model

def make_sampler(labels, power=0.5):
    # inverse sqrt frequency weights by default
    vals, counts = np.unique(labels, return_counts=True)
    freq = np.zeros(labels.max()+1, dtype=np.float64)
    freq[vals] = counts
    w = 1.0 / np.clip(freq, 1, None)**power
    weights = w[labels]
    return WeightedRandomSampler(weights=torch.as_tensor(weights, dtype=torch.float32), num_samples=len(labels), replacement=True)

def train_smoke(df_train, epochs=1, batch_size=32, img_size=384, max_rows=4096):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    torch.backends.cudnn.benchmark = True
    df_small = df_train.sample(n=min(max_rows, len(df_train)), random_state=42) if len(df_train) > max_rows else df_train.copy()
    ds = HerbariumDataset(df_small, mode='train', img_size=img_size)
    sampler = make_sampler(df_small['label'].values)
    dl = DataLoader(ds, batch_size=batch_size, sampler=sampler, num_workers=2, pin_memory=True)
    model = make_model('convnext_tiny', num_classes=num_classes)
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
    optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=0.02)
    scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())
    model.train()
    t0 = time.time()
    for ep in range(epochs):
        running = 0.0
        n = 0
        for bi, (x,y) in enumerate(dl):
            x = x.to(device, non_blocking=True)
            y = torch.as_tensor(y, device=device)
            optimizer.zero_grad(set_to_none=True)
            with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
                logits = model(x)
                loss = criterion(logits, y)
            scaler.scale(loss).step(optimizer)
            scaler.update()
            running += loss.item()*x.size(0)
            n += x.size(0)
            if (bi+1) % 50 == 0:
                elapsed = time.time()-t0
                print(f'Epoch {ep} Batch {bi+1}/{len(dl)} loss {running/max(n,1):.4f} elapsed {elapsed:.1f}s', flush=True)
        print(f'Epoch {ep} done. Avg loss {running/max(n,1):.4f}')
    return model

def infer_test(model, df_test, batch_size=64, img_size=384):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    torch.backends.cudnn.benchmark = True
    ds = HerbariumDataset(df_test, mode='test', img_size=img_size)
    dl = DataLoader(ds, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)
    model.eval()
    preds = []
    ids = []
    t0 = time.time()
    with torch.no_grad():
        for bi, (x, id_batch) in enumerate(dl):
            x = x.to(device, non_blocking=True)
            with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
                logits = model(x)
            pred_idx = torch.argmax(logits, dim=1).detach().cpu().numpy()
            preds.append(pred_idx)
            ids.extend(list(id_batch))
            if (bi+1) % 50 == 0:
                print(f'Infer batch {bi+1}/{len(dl)} elapsed {time.time()-t0:.1f}s')
    preds = np.concatenate(preds) if preds else np.array([], dtype=np.int64)
    pred_cids = idx2cid[preds] if len(preds)>0 else np.array([], dtype=np.int64)
    sub = pd.read_csv(base/'sample_submission.csv')
    id2pred = dict(zip(ids, pred_cids.tolist()))
    sub['Predicted'] = sub['Id'].astype(str).map(id2pred).fillna(int(df_train['category_id'].mode().iloc[0])).astype(int)
    return sub

print('CNN scaffold ready (PIL + torchvision transforms). Functions: train_smoke(), infer_test(). No training executed by default.')

/app/.pip-target/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Train rows: 1779953 Num classes: 64500


jpg counts → train: 1779953, test: 477806
CNN scaffold ready (torchvision transforms). Functions: train_smoke(), infer_test(). No training executed by default.


In [8]:
# Smoke-run CNN scaffold (no training) and write submission_dummy.csv (small test subset)
print('Starting CNN smoke run (no training) ...')
model = train_smoke(df_train, epochs=0, batch_size=8, img_size=224, max_rows=64)

# Use a tiny subset of test to avoid heavy inference; fill rest with global mode
df_te_small = df_test.sample(n=min(256, len(df_test)), random_state=42).copy()
sub_dummy = infer_test(model, df_te_small, batch_size=64, img_size=224)
sub_dummy.to_csv('submission_dummy.csv', index=False)
print('Wrote submission_dummy.csv (from small test subset + global mode fallback). Head:')
print(sub_dummy.head())
print('Done.')

Starting CNN smoke run (no training) ...


INFO:timm.models._builder:Loading pretrained weights from Hugging Face hub (timm/convnext_tiny.in12k_ft_in1k)


INFO:timm.models._hub:[timm/convnext_tiny.in12k_ft_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.


INFO:timm.models._builder:Missing keys (head.fc.weight, head.fc.bias) discovered while loading pretrained weights. This is expected if model is being adapted.


/tmp/ipykernel_94/654275732.py:100: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())


/tmp/ipykernel_94/654275732.py:134: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):


Wrote submission_dummy.csv (from small test subset + global mode fallback). Head:
   Id  Predicted
0   0      42811
1   1      42811
2   2      42811
3   3      42811
4   4      42811
Done.


In [9]:
# Smoothed metadata baseline: min-count gating on fallback chain
import json, pandas as pd, numpy as np
from pathlib import Path

base = Path('.')
train_meta = json.load(open(base/'train'/'metadata.json','r'))
test_meta = json.load(open(base/'test'/'metadata.json','r'))
ss = pd.read_csv(base/'sample_submission.csv')

df_img = pd.DataFrame(train_meta['images'])[['file_name','width','height','id']]
df_anno = pd.DataFrame(train_meta['annotations'])[['image_id','category_id']]
df_tr = df_anno.merge(df_img, left_on='image_id', right_on='id', how='left')

def parse_F0_F1(s):
    parts = s.split('/')
    f0 = parts[1] if len(parts) > 1 else ''
    f1 = parts[2] if len(parts) > 2 else ''
    return f0, f1

F0_F1 = df_tr['file_name'].map(parse_F0_F1)
df_tr['F0'] = [t[0] for t in F0_F1]
df_tr['F1'] = [t[1] for t in F0_F1]
df_tr['aspect'] = (df_tr['width'] / df_tr['height']).astype(float)
width_bins = [0, 600, 700, 800, 900, 1000, 1100, 2000]
height_bins = [0, 600, 700, 800, 900, 1000, 1100, 2000]
aspect_bins = [0, 0.6, 0.75, 0.9, 1.0, 1.1, 1.3, 2.5]
df_tr['wb'] = pd.cut(df_tr['width'], bins=width_bins, include_lowest=True).astype(str)
df_tr['hb'] = pd.cut(df_tr['height'], bins=height_bins, include_lowest=True).astype(str)
df_tr['ab'] = pd.cut(df_tr['aspect'], bins=aspect_bins, include_lowest=True).astype(str)

global_mode = int(df_tr['category_id'].mode().iloc[0])
print('Global mode:', global_mode)

def mode_map_thresh(df, keys, min_count=10):
    grp = df.groupby(keys)['category_id']
    counts = grp.size().rename('n')
    modal = grp.agg(lambda x: x.value_counts().idxmax())
    m = pd.concat([modal, counts], axis=1).reset_index()
    m = m[m['n'] >= min_count]
    # build dict: key_tuple -> modal category
    keycols = keys.copy()
    m['key'] = list(map(tuple, m[keycols].values.tolist()))
    return dict(zip(m['key'], m['category_id']))

min_n = 10
m_F0_F1_wb_hb = mode_map_thresh(df_tr, ['F0','F1','wb','hb'], min_n)
m_F0_F1_ab   = mode_map_thresh(df_tr, ['F0','F1','ab'], min_n)
m_F0_wb_hb   = mode_map_thresh(df_tr, ['F0','wb','hb'], min_n)
m_F0_ab      = mode_map_thresh(df_tr, ['F0','ab'], min_n)
m_F0_F1      = mode_map_thresh(df_tr, ['F0','F1'], min_n)
m_F0_wb      = mode_map_thresh(df_tr, ['F0','wb'], min_n)
m_F0_hb      = mode_map_thresh(df_tr, ['F0','hb'], min_n)
m_F0         = mode_map_thresh(df_tr, ['F0'], min_n)

df_te = pd.DataFrame(test_meta['images'])[['id','file_name','width','height']].copy()
F0_F1_te = df_te['file_name'].map(parse_F0_F1)
df_te['F0'] = [t[0] for t in F0_F1_te]
df_te['F1'] = [t[1] for t in F0_F1_te]
df_te['aspect'] = (df_te['width'] / df_te['height']).astype(float)
df_te['wb'] = pd.cut(df_te['width'], bins=width_bins, include_lowest=True).astype(str)
df_te['hb'] = pd.cut(df_te['height'], bins=height_bins, include_lowest=True).astype(str)
df_te['ab'] = pd.cut(df_te['aspect'], bins=aspect_bins, include_lowest=True).astype(str)

levels = [
    ('F0_F1_wb_hb', lambda r: m_F0_F1_wb_hb.get((r['F0'], r['F1'], r['wb'], r['hb']))),
    ('F0_F1_ab',    lambda r: m_F0_F1_ab.get((r['F0'], r['F1'], r['ab']))),
    ('F0_wb_hb',    lambda r: m_F0_wb_hb.get((r['F0'], r['wb'], r['hb']))),
    ('F0_ab',       lambda r: m_F0_ab.get((r['F0'], r['ab']))),
    ('F0_F1',       lambda r: m_F0_F1.get((r['F0'], r['F1']))),
    ('F0_wb',       lambda r: m_F0_wb.get((r['F0'], r['wb']))),
    ('F0_hb',       lambda r: m_F0_hb.get((r['F0'], r['hb']))),
    ('F0',          lambda r: m_F0.get((r['F0'],)))
]

pred = np.full(len(df_te), global_mode, dtype=np.int64)
covered = np.zeros(len(df_te), dtype=bool)

for name, fn in levels:
    if covered.all():
        break
    idx = np.where(~covered)[0]
    vals = [fn(df_te.iloc[i]) for i in idx]
    take = [v is not None for v in vals]
    if any(take):
        pred_idx = np.array(idx)[np.array(take)]
        pred_vals = np.array([int(v) for v in np.array(vals, dtype=object)[np.array(take)]])
        pred[pred_idx] = pred_vals
        covered[pred_idx] = True
    print(f'Level {name}: covered {covered.mean():.3f}')

sub = ss.copy()
id2pred = dict(zip(df_te['id'].astype(str), pred.tolist()))
sub['Predicted'] = sub['Id'].astype(str).map(id2pred).fillna(global_mode).astype(int)
sub.to_csv('submission.csv', index=False)
sub.to_csv('submission_f0_smoothed.csv', index=False)
print('Wrote submission.csv and submission_f0_smoothed.csv. Head:')
print(sub.head())

Global mode: 42811


Level F0_F1_wb_hb: covered 0.000


Level F0_F1_ab: covered 0.000


Level F0_wb_hb: covered 0.990
Level F0_ab: covered 0.991


Level F0_F1: covered 0.991
Level F0_wb: covered 0.991


Level F0_hb: covered 1.000
Level F0: covered 1.000


Wrote submission.csv and submission_f0_smoothed.csv. Head:
   Id  Predicted
0   0         23
1   1         23
2   2         23
3   3         23
4   4         23


In [10]:
# Per-F0 Naive Bayes baseline with Laplace smoothing and top-K classes
import json, pandas as pd, numpy as np, time
from pathlib import Path

base = Path('.')
train_meta = json.load(open(base/'train'/'metadata.json','r'))
test_meta = json.load(open(base/'test'/'metadata.json','r'))
ss = pd.read_csv(base/'sample_submission.csv')

# Prepare train features
df_img = pd.DataFrame(train_meta['images'])[['file_name','width','height','id']]
df_anno = pd.DataFrame(train_meta['annotations'])[['image_id','category_id']]
df_tr = df_anno.merge(df_img, left_on='image_id', right_on='id', how='left')

def parse_F0_F1(s):
    parts = s.split('/')
    f0 = parts[1] if len(parts) > 1 else ''
    f1 = parts[2] if len(parts) > 2 else ''
    return f0, f1

F0_F1 = df_tr['file_name'].map(parse_F0_F1)
df_tr['F0'] = [t[0] for t in F0_F1]
df_tr['F1'] = [t[1] for t in F0_F1]
df_tr['aspect'] = (df_tr['width'] / df_tr['height']).astype(float)
width_bins = [0, 600, 700, 800, 900, 1000, 1100, 2000]
height_bins = [0, 600, 700, 800, 900, 1000, 1100, 2000]
aspect_bins = [0, 0.6, 0.75, 0.9, 1.0, 1.1, 1.3, 2.5]
df_tr['wb'] = pd.cut(df_tr['width'], bins=width_bins, include_lowest=True).astype(str)
df_tr['hb'] = pd.cut(df_tr['height'], bins=height_bins, include_lowest=True).astype(str)
df_tr['ab'] = pd.cut(df_tr['aspect'], bins=aspect_bins, include_lowest=True).astype(str)

# Fixed bin label universes (consistent mapping train/test)
wb_labels = pd.cut(pd.Series(width_bins[:-1]) + 1e-6, bins=width_bins, include_lowest=True).astype(str).unique().tolist()
hb_labels = pd.cut(pd.Series(height_bins[:-1]) + 1e-6, bins=height_bins, include_lowest=True).astype(str).unique().tolist()
ab_labels = pd.cut(pd.Series(aspect_bins[:-1]) + 1e-6, bins=aspect_bins, include_lowest=True).astype(str).unique().tolist()
wb2i = {l:i for i,l in enumerate(wb_labels)}
hb2i = {l:i for i,l in enumerate(hb_labels)}
ab2i = {l:i for i,l in enumerate(ab_labels)}
Bwb, Bhb, Bab = len(wb2i), len(hb2i), len(ab2i)

global_mode = int(df_tr['category_id'].mode().iloc[0])
print('Global mode:', global_mode)

# Build per-F0 NB parameters
alpha = 1.0
topK = 200
models = {}  # F0 -> dict with classes, log_prior, logPwb[Bwb,K], logPhb[Bhb,K], logPab[Bab,K]
t0 = time.time()
for f0, g in df_tr.groupby('F0', sort=False):
    cls_counts = g['category_id'].value_counts()
    classes = cls_counts.head(topK).index.values.astype(np.int64)
    counts = cls_counts.head(topK).values.astype(np.float64)
    C = len(classes)
    if C == 0:
        continue
    prior = (counts + alpha) / (counts.sum() + alpha * C)
    # Initialize count matrices with alpha for smoothing
    Cwb = np.full((Bwb, C), alpha, dtype=np.float64)
    Chb = np.full((Bhb, C), alpha, dtype=np.float64)
    Cab = np.full((Bab, C), alpha, dtype=np.float64)
    # Precompute bin indices for group
    gi_wb = g['wb'].map(wb2i).fillna(-1).astype(int).values
    gi_hb = g['hb'].map(hb2i).fillna(-1).astype(int).values
    gi_ab = g['ab'].map(ab2i).fillna(-1).astype(int).values
    gi_cls = g['category_id'].values
    # For each candidate class, accumulate bin counts
    cls2pos = {c:i for i,c in enumerate(classes)}
    for idx_row in range(len(g)):
        c = gi_cls[idx_row]
        j = cls2pos.get(c, None)
        if j is None:
            continue
        iw, ih, ia = gi_wb[idx_row], gi_hb[idx_row], gi_ab[idx_row]
        if iw >= 0: Cwb[iw, j] += 1.0
        if ih >= 0: Chb[ih, j] += 1.0
        if ia >= 0: Cab[ia, j] += 1.0
    # Normalize to probabilities and take logs
    log_prior = np.log(prior + 1e-12)
    logPwb = np.log(Cwb / Cwb.sum(axis=0, keepdims=True))
    logPhb = np.log(Chb / Chb.sum(axis=0, keepdims=True))
    logPab = np.log(Cab / Cab.sum(axis=0, keepdims=True))
    models[f0] = dict(classes=classes, log_prior=log_prior, logPwb=logPwb, logPhb=logPhb, logPab=logPab)
    if len(models) % 50 == 0:
        print(f'Built NB for {len(models)} F0 shards, elapsed {time.time()-t0:.1f}s', flush=True)
print(f'Total F0 shards modeled: {len(models)} in {time.time()-t0:.1f}s')

# Prepare test features
df_te = pd.DataFrame(test_meta['images'])[['id','file_name','width','height']].copy()
F0_F1_te = df_te['file_name'].map(parse_F0_F1)
df_te['F0'] = [t[0] for t in F0_F1_te]
df_te['aspect'] = (df_te['width'] / df_te['height']).astype(float)
df_te['wb'] = pd.cut(df_te['width'], bins=width_bins, include_lowest=True).astype(str)
df_te['hb'] = pd.cut(df_te['height'], bins=height_bins, include_lowest=True).astype(str)
df_te['ab'] = pd.cut(df_te['aspect'], bins=aspect_bins, include_lowest=True).astype(str)

# Inference per F0
preds = np.full(len(df_te), global_mode, dtype=np.int64)
t1 = time.time()
for f0, g in df_te.groupby('F0', sort=False):
    idx = g.index.values
    m = models.get(f0, None)
    if m is None:
        continue
    classes = m['classes']  # (K,)
    K = classes.shape[0]
    logp = np.tile(m['log_prior'][None, :], (len(g), 1))  # (n,K)
    iw = g['wb'].map(wb2i).fillna(-1).astype(int).values
    ih = g['hb'].map(hb2i).fillna(-1).astype(int).values
    ia = g['ab'].map(ab2i).fillna(-1).astype(int).values
    # Add likelihood terms where bins are known
    sel = iw >= 0
    if sel.any():
        logp[sel] += m['logPwb'][iw[sel], :]
    sel = ih >= 0
    if sel.any():
        logp[sel] += m['logPhb'][ih[sel], :]
    sel = ia >= 0
    if sel.any():
        logp[sel] += 0.5 * m['logPab'][ia[sel], :]
    jj = np.argmax(logp, axis=1)
    preds[idx] = classes[jj]
    if len(models) >= 1 and (len(idx) >= 20000):
        print(f'F0 {f0}: predicted {len(idx)} rows', flush=True)
print(f'Inference done in {time.time()-t1:.1f}s')

# Build submission
sub = ss.copy()
id2pred = dict(zip(df_te['id'].astype(str), preds.tolist()))
sub['Predicted'] = sub['Id'].astype(str).map(id2pred).fillna(global_mode).astype(int)
sub.to_csv('submission.csv', index=False)
sub.to_csv('submission_nb_f0.csv', index=False)
print('Wrote submission.csv and submission_nb_f0.csv. Head:')
print(sub.head())

Global mode: 42811


Built NB for 50 F0 shards, elapsed 0.6s


Built NB for 100 F0 shards, elapsed 0.9s


Built NB for 150 F0 shards, elapsed 1.1s


Built NB for 200 F0 shards, elapsed 1.4s


...

Built NB for 550 F0 shards, elapsed 3.0s


Built NB for 600 F0 shards, elapsed 3.2s


Total F0 shards modeled: 645 in 3.4s


Inference done in 0.7s


Wrote submission.csv and submission_nb_f0.csv. Head:
   Id  Predicted
0   0         23
1   1         23
2   2         23
3   3         23
4   4         23


In [11]:
# Tuned per-F0 Naive Bayes (topK=500, prior alpha=0.5, likelihood alpha=1.5, wb=1.0, hb=1.0, ab=0.3)
import json, pandas as pd, numpy as np, time
from pathlib import Path

base = Path('.')
train_meta = json.load(open(base/'train'/'metadata.json','r'))
test_meta = json.load(open(base/'test'/'metadata.json','r'))
ss = pd.read_csv(base/'sample_submission.csv')

df_img = pd.DataFrame(train_meta['images'])[['file_name','width','height','id']]
df_anno = pd.DataFrame(train_meta['annotations'])[['image_id','category_id']]
df_tr = df_anno.merge(df_img, left_on='image_id', right_on='id', how='left')

def parse_F0_F1(s):
    parts = s.split('/')
    f0 = parts[1] if len(parts) > 1 else ''
    f1 = parts[2] if len(parts) > 2 else ''
    return f0, f1

F0_F1 = df_tr['file_name'].map(parse_F0_F1)
df_tr['F0'] = [t[0] for t in F0_F1]
df_tr['aspect'] = (df_tr['width'] / df_tr['height']).astype(float)
width_bins = [0, 600, 700, 800, 900, 1000, 1100, 2000]
height_bins = [0, 600, 700, 800, 900, 1000, 1100, 2000]
aspect_bins = [0, 0.6, 0.75, 0.9, 1.0, 1.1, 1.3, 2.5]
df_tr['wb'] = pd.cut(df_tr['width'], bins=width_bins, include_lowest=True).astype(str)
df_tr['hb'] = pd.cut(df_tr['height'], bins=height_bins, include_lowest=True).astype(str)
df_tr['ab'] = pd.cut(df_tr['aspect'], bins=aspect_bins, include_lowest=True).astype(str)

wb_labels = pd.cut(pd.Series(width_bins[:-1]) + 1e-6, bins=width_bins, include_lowest=True).astype(str).unique().tolist()
hb_labels = pd.cut(pd.Series(height_bins[:-1]) + 1e-6, bins=height_bins, include_lowest=True).astype(str).unique().tolist()
ab_labels = pd.cut(pd.Series(aspect_bins[:-1]) + 1e-6, bins=aspect_bins, include_lowest=True).astype(str).unique().tolist()
wb2i = {l:i for i,l in enumerate(wb_labels)}
hb2i = {l:i for i,l in enumerate(hb_labels)}
ab2i = {l:i for i,l in enumerate(ab_labels)}
Bwb, Bhb, Bab = len(wb2i), len(hb2i), len(ab2i)

global_mode = int(df_tr['category_id'].mode().iloc[0])
print('Global mode:', global_mode)

alpha_prior = 0.5
alpha_like = 1.5
topK = 500
w_wb, w_hb, w_ab = 1.0, 1.0, 0.3

models = {}
t0 = time.time()
for f0, g in df_tr.groupby('F0', sort=False):
    cls_counts = g['category_id'].value_counts()
    classes = cls_counts.head(topK).index.values.astype(np.int64)
    counts = cls_counts.head(topK).values.astype(np.float64)
    C = len(classes)
    if C == 0:
        continue
    prior = (counts + alpha_prior) / (counts.sum() + alpha_prior * C)
    Cwb = np.full((Bwb, C), alpha_like, dtype=np.float64)
    Chb = np.full((Bhb, C), alpha_like, dtype=np.float64)
    Cab = np.full((Bab, C), alpha_like, dtype=np.float64)
    gi_wb = g['wb'].map(wb2i).fillna(-1).astype(int).values
    gi_hb = g['hb'].map(hb2i).fillna(-1).astype(int).values
    gi_ab = g['ab'].map(ab2i).fillna(-1).astype(int).values
    gi_cls = g['category_id'].values
    cls2pos = {c:i for i,c in enumerate(classes)}
    for idx_row in range(len(g)):
        j = cls2pos.get(gi_cls[idx_row], None)
        if j is None:
            continue
        iw, ih, ia = gi_wb[idx_row], gi_hb[idx_row], gi_ab[idx_row]
        if iw >= 0: Cwb[iw, j] += 1.0
        if ih >= 0: Chb[ih, j] += 1.0
        if ia >= 0: Cab[ia, j] += 1.0
    log_prior = np.log(prior + 1e-12)
    logPwb = np.log(Cwb / Cwb.sum(axis=0, keepdims=True))
    logPhb = np.log(Chb / Chb.sum(axis=0, keepdims=True))
    logPab = np.log(Cab / Cab.sum(axis=0, keepdims=True))
    models[f0] = dict(classes=classes, log_prior=log_prior, logPwb=logPwb, logPhb=logPhb, logPab=logPab)
    if len(models) % 50 == 0:
        print(f'Built tuned NB for {len(models)} F0 shards, elapsed {time.time()-t0:.1f}s', flush=True)
print(f'Total F0 shards modeled (tuned): {len(models)} in {time.time()-t0:.1f}s')

df_te = pd.DataFrame(test_meta['images'])[['id','file_name','width','height']].copy()
F0_F1_te = df_te['file_name'].map(parse_F0_F1)
df_te['F0'] = [t[0] for t in F0_F1_te]
df_te['aspect'] = (df_te['width'] / df_te['height']).astype(float)
df_te['wb'] = pd.cut(df_te['width'], bins=width_bins, include_lowest=True).astype(str)
df_te['hb'] = pd.cut(df_te['height'], bins=height_bins, include_lowest=True).astype(str)
df_te['ab'] = pd.cut(df_te['aspect'], bins=aspect_bins, include_lowest=True).astype(str)

preds = np.full(len(df_te), global_mode, dtype=np.int64)
t1 = time.time()
for f0, g in df_te.groupby('F0', sort=False):
    idx = g.index.values
    m = models.get(f0, None)
    if m is None:
        continue
    classes = m['classes']
    K = classes.shape[0]
    logp = np.tile(m['log_prior'][None, :], (len(g), 1))
    iw = g['wb'].map(wb2i).fillna(-1).astype(int).values
    ih = g['hb'].map(hb2i).fillna(-1).astype(int).values
    ia = g['ab'].map(ab2i).fillna(-1).astype(int).values
    sel = iw >= 0
    if sel.any():
        logp[sel] += w_wb * m['logPwb'][iw[sel], :]
    sel = ih >= 0
    if sel.any():
        logp[sel] += w_hb * m['logPhb'][ih[sel], :]
    sel = ia >= 0
    if sel.any():
        logp[sel] += w_ab * m['logPab'][ia[sel], :]
    jj = np.argmax(logp, axis=1)
    preds[idx] = classes[jj]
print(f'Tuned NB inference done in {time.time()-t1:.1f}s')

sub = ss.copy()
id2pred = dict(zip(df_te['id'].astype(str), preds.tolist()))
sub['Predicted'] = sub['Id'].astype(str).map(id2pred).fillna(global_mode).astype(int)
sub.to_csv('submission.csv', index=False)
sub.to_csv('submission_nb_f0_tuned.csv', index=False)
print('Wrote submission.csv and submission_nb_f0_tuned.csv. Head:')
print(sub.head())

Global mode: 42811


Built tuned NB for 50 F0 shards, elapsed 0.6s


Built tuned NB for 100 F0 shards, elapsed 0.9s


Built tuned NB for 150 F0 shards, elapsed 1.1s


Built tuned NB for 200 F0 shards, elapsed 1.4s


...

Built tuned NB for 550 F0 shards, elapsed 3.0s


Built tuned NB for 600 F0 shards, elapsed 3.2s


Total F0 shards modeled (tuned): 645 in 3.4s


Tuned NB inference done in 0.7s


Wrote submission.csv and submission_nb_f0_tuned.csv. Head:
   Id  Predicted
0   0         23
1   1         23
2   2         23
3   3         23
4   4         23


In [ ]:
# Blend NB(tuned) with smoothed fallback using per-F0 class prior as tie-break
import json, pandas as pd, numpy as np
from pathlib import Path

base = Path('.')
train_meta = json.load(open(base/'train'/'metadata.json','r'))
test_meta = json.load(open(base/'test'/'metadata.json','r'))

# Load candidate submissions
sub_nb = pd.read_csv('submission_nb_f0_tuned.csv') if Path('submission_nb_f0_tuned.csv').exists() else pd.read_csv('submission_nb_f0.csv')
sub_sm = pd.read_csv('submission_f0_smoothed.csv') if Path('submission_f0_smoothed.csv').exists() else pd.read_csv('submission_f0_ext.csv')

# Build per-F0 class prior from train
df_img = pd.DataFrame(train_meta['images'])[['file_name','width','height','id']]
df_anno = pd.DataFrame(train_meta['annotations'])[['image_id','category_id']]
df_tr = df_anno.merge(df_img, left_on='image_id', right_on='id', how='left')
def parse_F0(s):
    parts = s.split('/')
    return parts[1] if len(parts) > 1 else ''
df_tr['F0'] = df_tr['file_name'].map(parse_F0)
prior_f0 = df_tr.groupby(['F0','category_id']).size().rename('cnt').reset_index()
prior_f0['key'] = list(zip(prior_f0['F0'], prior_f0['category_id']))
prior_map = prior_f0.set_index('key')['cnt'].to_dict()

# Test F0 per Id
df_te = pd.DataFrame(test_meta['images'])[['id','file_name']].copy()
df_te['F0'] = df_te['file_name'].map(parse_F0)
id2f0 = dict(zip(df_te['id'].astype(str), df_te['F0']))

# Align and blend
sub = sub_nb.merge(sub_sm, on='Id', how='left', suffixes=('_nb','_sm'))
sub['Predicted_sm'].fillna(sub['Predicted_nb'], inplace=True)
def choose_row(r):
    f0 = id2f0.get(str(r['Id']), '')
    c_nb = int(r['Predicted_nb'])
    c_sm = int(r['Predicted_sm'])
    if c_nb == c_sm:
        return c_nb
    cnt_nb = prior_map.get((f0, c_nb), 0)
    cnt_sm = prior_map.get((f0, c_sm), 0)
    # prefer higher prior within F0; tie -> NB
    return c_nb if cnt_nb >= cnt_sm else c_sm

sub['Predicted'] = sub.apply(choose_row, axis=1).astype(int)
out = sub[['Id','Predicted']].copy()
out.to_csv('submission.csv', index=False)
out.to_csv('submission_blend_nb_smoothed.csv', index=False)
print('Wrote submission.csv and submission_blend_nb_smoothed.csv. Head:')
print(out.head())

In [17]:
# Main CNN training run per expert plan (ConvNeXtV2-Base @384, steps-based, EMA, AMP, class-balanced sampler) - updates-based scheduler
import os, math, time, random, json, gc
from pathlib import Path
import numpy as np
import pandas as pd
from PIL import Image
from sklearn.metrics import f1_score
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import torchvision.transforms as T
import timm
from timm.utils import ModelEmaV2
from timm.data.mixup import Mixup
from timm.loss import SoftTargetCrossEntropy

base = Path('.')
train_dir = base/'train'
test_dir = base/'test'

# Reuse df_train, df_test, idx2cid, cid2idx, num_classes from earlier cell if present; otherwise, build them
try:
    df_train
    df_test
    idx2cid
    cid2idx
    num_classes
except NameError:
    train_meta = json.load(open(train_dir/'metadata.json','r'))
    test_meta = json.load(open(test_dir/'metadata.json','r'))
    df_img = pd.DataFrame(train_meta['images'])[['id','file_name','width','height']]
    df_anno = pd.DataFrame(train_meta['annotations'])[['image_id','category_id']]
    df_train = df_anno.merge(df_img, left_on='image_id', right_on='id', how='left')
    df_train['path'] = df_train['file_name'].apply(lambda s: str(train_dir / s))
    df_test = pd.DataFrame(test_meta['images'])[['id','file_name','width','height']].copy()
    df_test['path'] = df_test['file_name'].apply(lambda s: str(test_dir / s))
    unique_cids = sorted(df_train['category_id'].unique())
    cid2idx = {c:i for i,c in enumerate(unique_cids)}
    idx2cid = np.array(unique_cids, dtype=np.int64)
    df_train['label'] = df_train['category_id'].map(cid2idx).astype(np.int64)
    num_classes = len(unique_cids)
print('Train rows:', len(df_train), 'Num classes:', num_classes)

def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = False
    torch.backends.cudnn.benchmark = True
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True

class HerbariumDataset(Dataset):
    def __init__(self, df, mode='train', img_size=384):
        self.df = df.reset_index(drop=True)
        self.mode = mode
        self.img_size = img_size
        self.tf_train = T.Compose([
            T.RandomResizedCrop(img_size, scale=(0.7,1.0), ratio=(0.75,1.33)),
            T.RandomHorizontalFlip(p=0.5),
            T.ColorJitter(0.1,0.1,0.1,0.05),
            T.RandomRotation(degrees=15),
            T.ToTensor(),
            T.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225)),
        ])
        self.tf_val = T.Compose([
            T.Resize(img_size, interpolation=T.InterpolationMode.BICUBIC),
            T.CenterCrop(img_size),
            T.ToTensor(),
            T.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225)),
        ])
    def __len__(self):
        return len(self.df)
    def _load_image(self, path):
        try:
            if os.path.exists(path):
                with Image.open(path) as im:
                    return im.convert('RGB')
        except Exception:
            pass
        return Image.new('RGB', (self.img_size, self.img_size), (0,0,0))
    def __getitem__(self, idx):
        r = self.df.iloc[idx]
        img = self._load_image(r['path'])
        if self.mode == 'train':
            img = self.tf_train(img)
            return img, int(r['label'])
        elif self.mode == 'val':
            img = self.tf_val(img)
            return img, int(r['label'])
        else:
            img = self.tf_val(img)
            return img, str(r['id'])

def make_sampler(labels, power=0.5):
    vals, counts = np.unique(labels, return_counts=True)
    max_label = int(labels.max()) if len(labels)>0 else 0
    freq = np.zeros(max_label+1, dtype=np.float64)
    freq[vals] = counts
    w = 1.0 / np.clip(freq, 1, None)**power
    weights = w[labels]
    return WeightedRandomSampler(weights=torch.as_tensor(weights, dtype=torch.float32), num_samples=len(labels), replacement=True)

def macro_f1_from_logits(logits, y_true):
    y_pred = logits.argmax(1)
    return f1_score(y_true, y_pred, average='macro', zero_division=0)

def top1_acc_from_logits(logits, y_true):
    with torch.no_grad():
        pred = logits.argmax(1)
        return (pred == y_true).float().mean().item()

def make_val_split_min1_train(df, val_frac=0.05, seed=42):
    rng = np.random.default_rng(seed)
    by_class = df.groupby('label').indices
    val_indices = []
    p = val_frac
    for lbl, idxs in by_class.items():
        idxs = np.array(list(idxs))
        n = idxs.size
        if n <= 1:
            continue
        if rng.random() < p:
            choice = int(rng.choice(idxs))
            val_indices.append(choice)
    target_val = int(len(df) * val_frac)
    if len(val_indices) < target_val:
        need = target_val - len(val_indices)
        candidates = []
        for lbl, idxs in by_class.items():
            idxs = np.array(list(idxs))
            if idxs.size >= 3:
                candidates.append(int(idxs[0]))
        if candidates:
            extra = rng.choice(candidates, size=min(need, len(candidates)), replace=False)
            val_indices.extend(list(map(int, extra)))
    val_set = set(val_indices)
    va_idx = np.array([i for i in range(len(df)) if i in val_set], dtype=np.int64)
    tr_idx = np.array([i for i in range(len(df)) if i not in val_set], dtype=np.int64)
    tr_labels = set(df.iloc[tr_idx]['label'].unique().tolist())
    for lbl, idxs in by_class.items():
        if lbl not in tr_labels:
            idxs = list(idxs)
            moved = False
            for j in idxs:
                if j in val_set:
                    val_set.remove(j)
                    moved = True
                    break
            if moved:
                tr_labels.add(lbl)
    va_idx = np.array(sorted(list(val_set)), dtype=np.int64)
    tr_idx = np.array([i for i in range(len(df)) if i not in val_set], dtype=np.int64)
    return tr_idx, va_idx

def freeze_backbone_unfreeze_head(model):
    for n, p in model.named_parameters():
        p.requires_grad = ('head' in n)

def unfreeze_all(model):
    for p in model.parameters():
        p.requires_grad = True

def train_main(
    backbone='convnextv2_base',
    img_size=384,
    batch_size=32,
    eff_batch=128,
    updates_total=8_000,
    warmup_updates=300,
    lr_base=3e-4,
    weight_decay=0.02,
    seed=42,
    mixup_alpha=0.2,
    cutmix_alpha=0.2,
    mix_prob=0.8,
    val_frac=0.05,
    ckpt_dir='ckpts_main',
    head_warmup_updates=600,
    lr_head=3e-3
):
    seed_everything(seed)
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    os.makedirs(ckpt_dir, exist_ok=True)

    # Split
    tr_idx, va_idx = make_val_split_min1_train(df_train, val_frac=val_frac, seed=seed)
    dtr = df_train.iloc[tr_idx].reset_index(drop=True)
    dva = df_train.iloc[va_idx].reset_index(drop=True)
    print(f'Train/Val sizes: {len(dtr)}/{len(dva)} | classes in train: {dtr.label.nunique()}')

    ds_tr = HerbariumDataset(dtr, mode='train', img_size=img_size)
    ds_va = HerbariumDataset(dva, mode='val', img_size=img_size)
    sampler = make_sampler(dtr['label'].values, power=0.5)
    num_workers = 8
    dl_tr = DataLoader(ds_tr, batch_size=batch_size, sampler=sampler, pin_memory=True, num_workers=num_workers, persistent_workers=True, prefetch_factor=2)
    dl_va = DataLoader(ds_va, batch_size=max(64, batch_size), shuffle=False, pin_memory=True, num_workers=num_workers, persistent_workers=True, prefetch_factor=2)

    model = timm.create_model(backbone, pretrained=True, num_classes=num_classes)
    if hasattr(model, 'set_grad_checkpointing'):
        try:
            model.set_grad_checkpointing(True)
        except Exception:
            pass
    model.to(device)
    model.to(memory_format=torch.channels_last)

    ema = ModelEmaV2(model, decay=0.999, device=device)

    accum_steps = max(1, eff_batch // batch_size)
    scaled_lr = lr_base * ((batch_size * accum_steps) / 256.0)

    # Optimizers: start with head-only
    freeze_backbone_unfreeze_head(model)
    head_params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.AdamW(head_params, lr=lr_head, weight_decay=weight_decay)

    def cosine_lr(u):
        if u < warmup_updates:
            return (u + 1) / max(1, warmup_updates)
        t = (u - warmup_updates) / max(1, updates_total - warmup_updates)
        t = min(1.0, max(0.0, t))
        return 0.5 * (1 + math.cos(math.pi * t))

    mixup_fn = Mixup(mixup_alpha=mixup_alpha, cutmix_alpha=cutmix_alpha, prob=mix_prob, switch_prob=0.5, mode='batch', label_smoothing=0.1, num_classes=num_classes)
    criterion_soft = SoftTargetCrossEntropy()
    criterion_hard = nn.CrossEntropyLoss(label_smoothing=0.1)
    criterion_val = nn.CrossEntropyLoss(reduction='mean')

    scaler = torch.amp.GradScaler('cuda', enabled=torch.cuda.is_available())

    best_f1 = -1.0
    best_path = None
    micro_step = 0
    global_step = 0
    running_loss = 0.0
    samples_seen = 0
    last_train_top1 = 0.0
    t0 = time.time()

    model.train()
    while global_step < updates_total:
        for it, (x, y) in enumerate(dl_tr):
            if global_step >= updates_total:
                break

            # Unfreeze and switch optimizer after head warmup
            if global_step == head_warmup_updates:
                unfreeze_all(model)
                optimizer = torch.optim.AdamW(model.parameters(), lr=scaled_lr, weight_decay=weight_decay)

                # Reset EMA to keep tracking post-unfreeze weights smoothly
                # Note: EMA state continues; no reset of ema needed.

            x = x.to(device, non_blocking=True).to(memory_format=torch.channels_last)
            y = y.to(device, non_blocking=True)

            # LR policy: head phase uses fixed lr_head; after unfreeze use cosine schedule
            if global_step >= head_warmup_updates:
                cur_lr = scaled_lr * cosine_lr(global_step)
                for pg in optimizer.param_groups:
                    pg['lr'] = cur_lr
            else:
                for pg in optimizer.param_groups:
                    pg['lr'] = lr_head

            # Mixup OFF during head-only phase and during last ~1500 updates
            use_mix = (mixup_fn is not None) and (global_step >= head_warmup_updates) and (global_step < max(0, updates_total - 1_500))
            if use_mix:
                x, y_mix = mixup_fn(x, y)

            with torch.amp.autocast('cuda', enabled=torch.cuda.is_available()):
                logits = model(x)
                loss = criterion_soft(logits, y_mix) if use_mix else criterion_hard(logits, y)

            # Train mini-batch top-1 (sanity); compute against hard labels
            last_train_top1 = top1_acc_from_logits(logits.detach(), y)

            loss = loss / accum_steps
            scaler.scale(loss).backward()
            micro_step += 1

            if (micro_step % accum_steps) == 0:
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad(set_to_none=True)
                ema.update(model)
                global_step += 1

            running_loss += loss.item() * x.size(0) * accum_steps
            samples_seen += x.size(0)

            if (micro_step % 200) == 0:
                elapsed = time.time() - t0
                avg_loss = running_loss / max(1, samples_seen)
                cur_lr_print = optimizer.param_groups[0]['lr']
                print(f'update {global_step}/{updates_total} | micro {micro_step} | avg_loss {avg_loss:.4f} | lr {cur_lr_print:.2e} | train_top1 {last_train_top1*100:.2f}% | elapsed {elapsed/60:.1f}m', flush=True)

            # Validation cadence: every 500 updates until 2k, then every 1k
            need_val = False
            if global_step > 0 and global_step < 2000 and (global_step % 500 == 0):
                need_val = True
            elif global_step >= 2000 and (global_step % 1000 == 0):
                need_val = True
            if need_val or global_step >= updates_total:
                val_f1, val_top1, val_loss = evaluate(model, ema, dl_va, device, criterion_val)
                is_best = val_f1 > best_f1
                best_f1 = max(best_f1, val_f1)
                ckpt_path = Path(ckpt_dir)/f'model_upd{global_step}_f1{val_f1:.5f}.pt'
                save_checkpoint(model, ema, optimizer, global_step, best_f1, ckpt_path)
                if is_best:
                    best_path = ckpt_path
                print(f'Validation @update {global_step}: macro-F1={val_f1:.5f} | top1={val_top1*100:.2f}% | loss={val_loss:.4f} | best={best_f1:.5f} | saved={ckpt_path.name}', flush=True)

    print('Training done. Best ckpt:', best_path)
    return dict(best_ckpt=str(best_path) if best_path else None, best_f1=best_f1)

def evaluate(model, ema, dl_va, device, criterion_val):
    # Use EMA model directly, compute F1, top1, and CE loss
    ema_model = ema.module
    was_training = ema_model.training
    ema_model.eval()
    y_preds = []
    y_trues = []
    running_loss = 0.0
    n_items = 0
    with torch.no_grad():
        for x, y in dl_va:
            x = x.to(device, non_blocking=True).to(memory_format=torch.channels_last)
            y = y.to(device, non_blocking=True)
            with torch.amp.autocast('cuda', enabled=torch.cuda.is_available()):
                logits = ema_model(x)
                loss = criterion_val(logits, y)
            running_loss += loss.item() * x.size(0)
            n_items += x.size(0)
            y_preds.append(torch.argmax(logits, dim=1).detach().cpu().numpy())
            y_trues.append(y.detach().cpu().numpy())
    y_pred = np.concatenate(y_preds) if y_preds else np.array([], dtype=np.int64)
    y_true = np.concatenate(y_trues) if y_trues else np.array([], dtype=np.int64)
    f1 = f1_score(y_true, y_pred, average='macro', zero_division=0) if len(y_true) else 0.0
    top1 = (y_pred == y_true).mean() if len(y_true) else 0.0
    val_loss = running_loss / max(1, n_items)
    if was_training:
        ema_model.train()
    return float(f1), float(top1), float(val_loss)

def save_checkpoint(model, ema, optimizer, step, best_f1, path):
    state = {
        'model': model.state_dict(),
        'ema': ema.state_dict(),
        'optimizer': optimizer.state_dict(),
        'step': step,
        'best_f1': best_f1,
    }
    torch.save(state, path)

print('Launching updates-based training with head-only warmup: convnextv2_base @384 | eff_batch=128 | updates_total=8k ...')
train_summary = train_main(
    backbone='convnextv2_base',
    img_size=384,
    batch_size=32,
    eff_batch=128,
    updates_total=8_000,
    warmup_updates=300,
    lr_base=3e-4,
    weight_decay=0.02,
    seed=42,
    mixup_alpha=0.2,
    cutmix_alpha=0.2,
    mix_prob=0.8,
    val_frac=0.05,
    ckpt_dir='ckpts_main',
    head_warmup_updates=600,
    lr_head=3e-3
)
print('Train summary:', train_summary)

Train rows: 1779953 Num classes: 64500
Launching updates-based training with head-only warmup: convnextv2_base @384 | eff_batch=128 | updates_total=8k ...


Train/Val sizes: 1726076/53877 | classes in train: 64500


INFO:timm.models._builder:Loading pretrained weights from Hugging Face hub (timm/convnextv2_base.fcmae_ft_in22k_in1k)


INFO:timm.models._hub:[timm/convnextv2_base.fcmae_ft_in22k_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.


INFO:timm.models._builder:Missing keys (head.fc.weight, head.fc.bias) discovered while loading pretrained weights. This is expected if model is being adapted.


...

/app/.pip-target/sklearn/metrics/_classification.py:98: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  type_true = type_of_target(y_true, input_name="y_true")
/app/.pip-target/sklearn/utils/multiclass.py:79: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  ys_types = set(type_of_target(x) for x in ys)
/app/.pip-target/sklearn/metrics/_classification.py:98: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  type_true = type_of_target(y_true, input_name="y_true")
/app/.pip-target/sklearn/utils/multiclass.py:79: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classifica

Validation @update 5000: macro-F1=0.07740 | top1=14.51% | loss=7.1650 | best=0.07740 | saved=model_upd5000_f10.07740.pt


/app/.pip-target/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


/app/.pip-target/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


/app/.pip-target/torch/utils/checkpoint.py:92: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


In [26]:
# Inference with optional 2x TTA (orig + hflip) and dual-tau outputs; adds optional per-F0 masking and per-F0 prior adjustment
import os, re, math, time, json
from pathlib import Path
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.transforms.functional as TF
import timm

def find_best_ckpt(ckpt_dir='ckpts_main'):
    ckpt_dir = Path(ckpt_dir)
    files = sorted(ckpt_dir.glob('model_upd*_f1*.pt'))
    if not files:
        print('No checkpoints found in', ckpt_dir)
        return None
    def parse_f1(p):
        m = re.search(r'_f1([0-9]+\.[0-9]+)\.pt$', p.name)
        return float(m.group(1)) if m else -1.0
    files = sorted(files, key=parse_f1, reverse=True)
    best = files[0]
    print('Selected best ckpt by filename f1:', best.name)
    return best

def load_model_from_ckpt(ckpt_path, backbone='convnextv2_base', num_classes=64500, device='cuda'):
    model = timm.create_model(backbone, pretrained=False, num_classes=num_classes)
    state = torch.load(ckpt_path, map_location='cpu')
    # Load base model weights first (non-strict to allow head adaptation)
    model.load_state_dict(state.get('model', {}), strict=False)
    # If EMA is present, load EMA and then copy its weights into the model
    try:
        if 'ema' in state:
            from timm.utils import ModelEmaV2
            ema = ModelEmaV2(model, decay=0.999)
            ema.load_state_dict(state['ema'], strict=False)
            model.load_state_dict(ema.module.state_dict(), strict=True)
            print('Loaded EMA weights into model')
    except Exception as e:
        print('EMA load skipped:', e)
    model.to(device)
    model.to(memory_format=torch.channels_last)
    model.eval()
    return model

def class_log_prior(labels, num_classes):
    counts = np.bincount(labels.astype(int), minlength=num_classes).astype(np.float64)
    return np.log(counts + 1.0)

def build_f0_maps(df_train, num_classes):
    # Parse F0 from file_name
    def parse_f0(s):
        parts = str(s).split('/')
        return parts[1] if len(parts) > 1 else ''
    f0 = df_train['file_name'].map(parse_f0)
    labels = df_train['label'].astype(int).values
    df_tmp = pd.DataFrame({'f0': f0, 'label': labels})
    # Per-F0 allowed classes
    mask_f0 = {}  # f0 -> (num_classes,) tensor with 0 for allowed and -inf for disallowed
    prior_f0 = {} # f0 -> (num_classes,) tensor of log prior
    for key, g in df_tmp.groupby('f0', sort=False):
        counts = np.bincount(g['label'].values, minlength=num_classes).astype(np.float64)
        allowed = (counts > 0).astype(np.float32)
        m = torch.full((num_classes,), -1e9, dtype=torch.float32)
        m[torch.from_numpy(allowed.astype(bool))] = 0.0
        mask_f0[key] = m
        prior = np.log(counts + 1.0)
        prior_f0[key] = torch.from_numpy(prior.astype(np.float32))
    return mask_f0, prior_f0

def tta2_logits(model, x):
    # 2x TTA: original + horizontal flip
    logits_list = []
    logits_list.append(model(x))
    logits_list.append(model(torch.flip(x, dims=[3])))
    return sum(logits_list) / len(logits_list)

def infer_test(
    df_test, idx2cid,
    backbone='convnextv2_base', img_size=384, batch_size=128, num_workers=6, ckpt_dir='ckpts_main',
    use_tta=False, tau_list=(None, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8), out_prefix='submission_cnn',
    use_f0_mask=False, use_per_f0_prior=False
):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    ckpt = find_best_ckpt(ckpt_dir)
    assert ckpt is not None, 'No checkpoint to load'
    num_classes = len(idx2cid)
    model = load_model_from_ckpt(ckpt, backbone=backbone, num_classes=num_classes, device=device)

    # Optional F0 maps
    mask_f0 = None
    prior_f0 = None
    if use_f0_mask or use_per_f0_prior:
        mask_f0, prior_f0 = build_f0_maps(df_train, num_classes)

    # id -> f0 map from df_test
    def parse_f0_from_path(s):
        parts = str(s).split('/')
        return parts[1] if len(parts) > 1 else ''
    id2f0 = dict(zip(df_test['id'].astype(str), df_test['file_name'].map(parse_f0_from_path)))

    ds = HerbariumDataset(df_test, mode='test', img_size=img_size)
    dl = DataLoader(ds, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True, persistent_workers=False, prefetch_factor=2)

    # Global adjustment once (fallback when not per-F0)
    base_adj_global = class_log_prior(df_train['label'].values, num_classes)
    base_adj_global = torch.from_numpy(base_adj_global).to(device=device, dtype=torch.float32)

    id_list = []
    pred_buffers = {tau: [] for tau in tau_list}

    t0 = time.time()
    with torch.no_grad():
        for bi, (x, ids) in enumerate(dl):
            x = x.to(device, non_blocking=True).to(memory_format=torch.channels_last)
            with torch.amp.autocast('cuda', enabled=torch.cuda.is_available()):
                logits = tta2_logits(model, x) if use_tta else model(x)
            # Per-sample F0 mask/prior stacks if enabled
            if use_f0_mask or use_per_f0_prior:
                f0_list = [id2f0.get(str(i), '') for i in ids]
            # Apply F0 mask (add large negative to disallowed classes)
            if use_f0_mask:
                m_list = [mask_f0.get(f0, None) for f0 in f0_list]
                m_stack = torch.stack([m if m is not None else torch.zeros(num_classes, dtype=torch.float32) for m in m_list], dim=0).to(device)
                logits = logits + m_stack
            # For each tau, adjust and argmax
            for tau in tau_list:
                if tau is None:
                    logits_adj = logits
                else:
                    if use_per_f0_prior:
                        adj_list = [prior_f0.get(f0, base_adj_global.cpu()).to(device) for f0 in (f0_list if (use_f0_mask or use_per_f0_prior) else [])]
                        if adj_list:
                            adj_stack = torch.stack(adj_list, dim=0).to(device)
                            logits_adj = logits - float(tau) * adj_stack
                        else:
                            logits_adj = logits - float(tau) * base_adj_global[None, :]
                    else:
                        logits_adj = logits - float(tau) * base_adj_global[None, :]
                pred_idx = torch.argmax(logits_adj, dim=1).detach().cpu().numpy()
                pred_buffers[tau].append(pred_idx)
            id_list.extend(list(ids))
            if (bi+1) % 50 == 0:
                print(f'Infer batch {bi+1}/{len(dl)} | elapsed {(time.time()-t0)/60:.1f}m', flush=True)

    outs = {}
    for tau in tau_list:
        pred_idx = np.concatenate(pred_buffers[tau]) if pred_buffers[tau] else np.array([], dtype=np.int64)
        pred_cids = idx2cid[pred_idx] if len(pred_idx)>0 else np.array([], dtype=np.int64)
        sub = pd.read_csv('sample_submission.csv')
        id2pred = dict(zip(id_list, pred_cids.tolist()))
        default_fill = int(sub['Predicted'].mode().iloc[0]) if 'Predicted' in sub.columns else 0
        sub['Predicted'] = sub['Id'].astype(str).map(id2pred).fillna(default_fill).astype(int)
        suffix = 'tauNone' if tau is None else 'tau' + str(tau).replace('.', '_')
        out_path = f'{out_prefix}_{suffix}.csv'
        sub.to_csv(out_path, index=False)
        print('Wrote', out_path, 'rows:', len(sub))
        outs[tau] = out_path
    return outs

print('Inference ready. Fast safety (no TTA, dual tau):')
print("infer_test(df_test, idx2cid, backbone='convnextv2_base', img_size=384, batch_size=128, num_workers=6, ckpt_dir='ckpts_main', use_tta=False, tau_list=(None,0.5), out_prefix='submission_cnn', use_f0_mask=False, use_per_f0_prior=False)")
print('For stronger run later, set use_tta=True (2x), use_per_f0_prior=True, and sweep tau in (None,0.2..0.8). Ensure you see: Loaded EMA weights into model.')

Inference ready. Fast safety (no TTA, dual tau):
infer_test(df_test, idx2cid, backbone='convnextv2_base', img_size=384, batch_size=128, num_workers=6, ckpt_dir='ckpts_main', use_tta=False, tau_list=(None,0.5), out_prefix='submission_cnn', use_f0_mask=False, use_per_f0_prior=False)
For stronger run later, set use_tta=True (2x), use_per_f0_prior=True, and sweep tau in (None,0.2..0.8). Ensure you see: Loaded EMA weights into model.


In [24]:
# Resume-capable training utilities (non-executing until called). Safe to add while main cell runs.
import os, math, time, json, random, numpy as np, torch, torch.nn as nn
from pathlib import Path

def load_resume_state(resume_path, model, ema, optimizer):
    start_step, best_f1 = 0, -1.0
    if resume_path is None or not os.path.exists(str(resume_path)):
        return start_step, best_f1, optimizer
    state = torch.load(resume_path, map_location='cpu')
    # Load model and EMA regardless of optimizer issues
    model.load_state_dict(state.get('model', {}), strict=False)
    if state.get('ema', None) is not None:
        try:
            ema.load_state_dict(state['ema'], strict=False)
        except Exception as e:
            print('EMA resume load skipped:', e)
    # Set resume metadata first
    start_step = int(state.get('step', 0))
    best_f1 = float(state.get('best_f1', -1.0))
    # Try optimizer resume; if it fails, keep training without it (do NOT reset step/best_f1)
    if 'optimizer' in state and optimizer is not None:
        try:
            optimizer.load_state_dict(state['optimizer'])
        except Exception as e:
            print('Optimizer state load failed; continuing without optimizer state:', e)
    print(f'Resumed from {resume_path} at step {start_step}, best_f1={best_f1:.5f}')
    return start_step, best_f1, optimizer

def train_main(
    backbone='convnextv2_base',
    img_size=384,
    batch_size=32,
    eff_batch=128,
    updates_total=8_000,
    warmup_updates=300,
    lr_base=3e-4,
    weight_decay=0.02,
    seed=42,
    mixup_alpha=0.2,
    cutmix_alpha=0.2,
    mix_prob=0.8,
    val_frac=0.05,
    ckpt_dir='ckpts_main',
    head_warmup_updates=600,
    lr_head=3e-3,
    resume_path=None
):
    # Reuse objects from earlier cells: df_train, HerbariumDataset, make_sampler, top1_acc_from_logits, evaluate, save_checkpoint, seed_everything
    seed_everything(seed)
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    os.makedirs(ckpt_dir, exist_ok=True)

    tr_idx, va_idx = make_val_split_min1_train(df_train, val_frac=val_frac, seed=seed)
    dtr = df_train.iloc[tr_idx].reset_index(drop=True)
    dva = df_train.iloc[va_idx].reset_index(drop=True)
    print(f'Train/Val sizes: {len(dtr)}/{len(dva)} | classes in train: {dtr.label.nunique()}')

    ds_tr = HerbariumDataset(dtr, mode='train', img_size=img_size)
    ds_va = HerbariumDataset(dva, mode='val', img_size=img_size)
    sampler = make_sampler(dtr['label'].values, power=0.5)
    num_workers = 8
    dl_tr = torch.utils.data.DataLoader(ds_tr, batch_size=batch_size, sampler=sampler, pin_memory=True, num_workers=num_workers, persistent_workers=True, prefetch_factor=2)
    dl_va = torch.utils.data.DataLoader(ds_va, batch_size=max(64, batch_size), shuffle=False, pin_memory=True, num_workers=num_workers, persistent_workers=True, prefetch_factor=2)

    model = timm.create_model(backbone, pretrained=True, num_classes=len(idx2cid))
    if hasattr(model, 'set_grad_checkpointing'):
        try: model.set_grad_checkpointing(True)
        except Exception: pass
    model.to(device)
    model.to(memory_format=torch.channels_last)

    ema = ModelEmaV2(model, decay=0.999, device=device)

    accum_steps = max(1, eff_batch // batch_size)
    scaled_lr = lr_base * ((batch_size * accum_steps) / 256.0)

    # Phase-aware optimizer init
    def freeze_backbone_unfreeze_head(m):
        for n, p in m.named_parameters():
            p.requires_grad = ('head' in n)
    def unfreeze_all(m):
        for p in m.parameters():
            p.requires_grad = True

    optimizer = None
    start_step = 0
    best_f1 = -1.0

    # Tentatively start in head-only mode; may be replaced after resume load
    freeze_backbone_unfreeze_head(model)
    head_params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.AdamW(head_params, lr=lr_head, weight_decay=weight_decay)

    # Try to resume
    start_step, best_f1, optimizer = load_resume_state(resume_path, model, ema, optimizer)

    # If we've passed warmup, switch to full optimizer
    if start_step >= head_warmup_updates:
        unfreeze_all(model)
        optimizer = torch.optim.AdamW(model.parameters(), lr=scaled_lr, weight_decay=weight_decay)

    def cosine_lr(u):
        if u < warmup_updates:
            return (u + 1) / max(1, warmup_updates)
        t = (u - warmup_updates) / max(1, updates_total - warmup_updates)
        t = min(1.0, max(0.0, t))
        return 0.5 * (1 + math.cos(math.pi * t))

    mixup_fn = Mixup(mixup_alpha=mixup_alpha, cutmix_alpha=cutmix_alpha, prob=mix_prob, switch_prob=0.5, mode='batch', label_smoothing=0.1, num_classes=len(idx2cid))
    criterion_soft = SoftTargetCrossEntropy()
    criterion_hard = nn.CrossEntropyLoss(label_smoothing=0.1)
    criterion_val = nn.CrossEntropyLoss(reduction='mean')
    scaler = torch.amp.GradScaler('cuda', enabled=torch.cuda.is_available())

    best_path = None
    micro_step = start_step * accum_steps
    global_step = start_step
    running_loss = 0.0
    samples_seen = 0
    last_train_top1 = 0.0
    t0 = time.time()

    model.train()
    while global_step < updates_total:
        for it, (x, y) in enumerate(dl_tr):
            if global_step >= updates_total:
                break

            if (global_step == head_warmup_updates) and any(not p.requires_grad for p in model.parameters()):
                unfreeze_all(model)
                optimizer = torch.optim.AdamW(model.parameters(), lr=scaled_lr, weight_decay=weight_decay)

            x = x.to(device, non_blocking=True).to(memory_format=torch.channels_last)
            y = y.to(device, non_blocking=True)

            if global_step >= head_warmup_updates:
                cur_lr = scaled_lr * cosine_lr(global_step)
                for pg in optimizer.param_groups: pg['lr'] = cur_lr
            else:
                for pg in optimizer.param_groups: pg['lr'] = lr_head

            use_mix = (mixup_fn is not None) and (global_step >= head_warmup_updates) and (global_step < max(0, updates_total - 1500))
            if use_mix:
                x, y_mix = mixup_fn(x, y)

            with torch.amp.autocast('cuda', enabled=torch.cuda.is_available()):
                logits = model(x)
                loss = criterion_soft(logits, y_mix) if use_mix else criterion_hard(logits, y)

            last_train_top1 = top1_acc_from_logits(logits.detach(), y)
            loss = loss / accum_steps
            scaler.scale(loss).backward()
            micro_step += 1

            if (micro_step % accum_steps) == 0:
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad(set_to_none=True)
                ema.update(model)
                global_step += 1

            running_loss += loss.item() * x.size(0) * accum_steps
            samples_seen += x.size(0)

            if (micro_step % 200) == 0:
                elapsed = time.time() - t0
                avg_loss = running_loss / max(1, samples_seen)
                cur_lr_print = optimizer.param_groups[0]['lr']
                print(f'update {global_step}/{updates_total} | micro {micro_step} | avg_loss {avg_loss:.4f} | lr {cur_lr_print:.2e} | train_top1 {last_train_top1*100:.2f}% | elapsed {elapsed/60:.1f}m', flush=True)

            need_val = False
            if global_step > 0 and global_step < 2000 and (global_step % 500 == 0):
                need_val = True
            elif global_step >= 2000 and (global_step % 1000 == 0):
                need_val = True
            if need_val or global_step >= updates_total:
                val_f1, val_top1, val_loss = evaluate(model, ema, dl_va, device, criterion_val)
                is_best = val_f1 > best_f1
                best_f1 = max(best_f1, val_f1)
                ckpt_path = Path(ckpt_dir)/f'model_upd{global_step}_f1{val_f1:.5f}.pt'
                save_checkpoint(model, ema, optimizer, global_step, best_f1, ckpt_path)
                if is_best:
                    best_path = ckpt_path
                print(f'Validation @update {global_step}: macro-F1={val_f1:.5f} | top1={val_top1*100:.2f}% | loss={val_loss:.4f} | best={best_f1:.5f} | saved={ckpt_path.name}', flush=True)

    print('Training done. Best ckpt:', best_path)
    return dict(best_ckpt=str(best_path) if best_path else None, best_f1=best_f1)

print('Resume-capable train_main(resume_path=...) defined. Use after pausing to safely resume from a saved ckpt.')

Resume-capable train_main(resume_path=...) defined. Use after pausing to safely resume from a saved ckpt.


In [19]:
# Safety submission inference: no TTA, dual tau
print('Running safety inference with best ckpt (no TTA, tau=None and 0.5) ...', flush=True)
outs = infer_test(
    df_test, idx2cid,
    backbone='convnextv2_base', img_size=384, batch_size=128, num_workers=6, ckpt_dir='ckpts_main',
    use_tta=False, tau_list=(None, 0.5), out_prefix='submission_cnn',
    use_f0_mask=False, use_per_f0_prior=False
)
print('Inference outputs:', outs, flush=True)

Running safety inference with best ckpt (no TTA, tau=None and 0.5) ...


Selected best ckpt by filename f1: model_upd5000_f10.07740.pt


/tmp/ipykernel_94/3039361667.py:28: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(ckpt_path, map_location='cpu')


EMA load skipped: 'ModelEmaV2' object has no attribute 'copy_to'


Infer batch 50/3733 | elapsed 0.9m


...

Infer batch 3600/3733 | elapsed 55.4m


Infer batch 3650/3733 | elapsed 56.2m


Infer batch 3700/3733 | elapsed 56.9m


Wrote submission_cnn_tauNone.csv rows: 477806


Wrote submission_cnn_tau0_5.csv rows: 477806
Inference outputs: {None: 'submission_cnn_tauNone.csv', 0.5: 'submission_cnn_tau0_5.csv'}


In [25]:
# Safety: set current best CNN submission as fallback, then resume training to 10k updates
import shutil, os, pandas as pd
best_fast_sub = 'submission_cnn_tau0_5.csv' if os.path.exists('submission_cnn_tau0_5.csv') else 'submission_cnn_tauNone.csv'
if os.path.exists(best_fast_sub):
    shutil.copyfile(best_fast_sub, 'submission.csv')
    print(f'Copied {best_fast_sub} -> submission.csv')
    try:
        print(pd.read_csv('submission.csv').head())
    except Exception as e:
        print('Readback head failed:', e)
else:
    print('No CNN submission found to copy as safety.')

print('Resuming training from upd5000 to 10k updates ...', flush=True)
train_summary = train_main(
  backbone='convnextv2_base', img_size=384,
  batch_size=32, eff_batch=128,
  updates_total=10_000, warmup_updates=300, lr_base=3e-4, weight_decay=0.02, seed=42,
  mixup_alpha=0.2, cutmix_alpha=0.2, mix_prob=0.8,
  val_frac=0.05, ckpt_dir='ckpts_main',
  head_warmup_updates=600, lr_head=3e-3,
  resume_path='ckpts_main/model_upd5000_f10.07740.pt'
)
print('Train summary:', train_summary)

Copied submission_cnn_tau0_5.csv -> submission.csv
   Id  Predicted
0   0      18454
1   1       5434
2   2      21267
3   3      10029
4   4      33711
Resuming training from upd5000 to 10k updates ...


Train/Val sizes: 1726076/53877 | classes in train: 64500


INFO:timm.models._builder:Loading pretrained weights from Hugging Face hub (timm/convnextv2_base.fcmae_ft_in22k_in1k)


INFO:timm.models._hub:[timm/convnextv2_base.fcmae_ft_in22k_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.


INFO:timm.models._builder:Missing keys (head.fc.weight, head.fc.bias) discovered while loading pretrained weights. This is expected if model is being adapted.


...

Validation @update 9000: macro-F1=0.11838 | top1=19.67% | loss=6.8369 | best=0.11838 | saved=model_upd9000_f10.11838.pt


/app/.pip-target/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


/app/.pip-target/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


/app/.pip-target/torch/utils/checkpoint.py:92: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


KeyboardInterrupt: 

In [27]:
# Strong inference (EMA, 2x TTA, per-F0 prior, tau sweep up to 0.8)
print('Running strong inference (EMA, 2x TTA, per-F0 prior, tau sweep) ...', flush=True)
outs_strong = infer_test(
    df_test, idx2cid,
    backbone='convnextv2_base', img_size=384,
    batch_size=128, num_workers=6, ckpt_dir='ckpts_main',
    use_tta=True,
    tau_list=(None, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8),
    out_prefix='submission_cnn_strong',
    use_f0_mask=False,
    use_per_f0_prior=True
)
print('Strong inference outputs:', outs_strong, flush=True)

Running strong inference (EMA, 2x TTA, per-F0 prior, tau sweep) ...


Selected best ckpt by filename f1: model_upd9000_f10.11838.pt


/tmp/ipykernel_94/825408699.py:28: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(ckpt_path, map_location='cpu')


Loaded EMA weights into model


Infer batch 50/3733 | elapsed 1.6m


...

Wrote submission_cnn_strong_tau0_4.csv rows: 477806


Wrote submission_cnn_strong_tau0_5.csv rows: 477806


Wrote submission_cnn_strong_tau0_6.csv rows: 477806


Wrote submission_cnn_strong_tau0_7.csv rows: 477806


Wrote submission_cnn_strong_tau0_8.csv rows: 477806
Strong inference outputs: {None: 'submission_cnn_strong_tauNone.csv', 0.2: 'submission_cnn_strong_tau0_2.csv', 0.3: 'submission_cnn_strong_tau0_3.csv', 0.4: 'submission_cnn_strong_tau0_4.csv', 0.5: 'submission_cnn_strong_tau0_5.csv', 0.6: 'submission_cnn_strong_tau0_6.csv', 0.7: 'submission_cnn_strong_tau0_7.csv', 0.8: 'submission_cnn_strong_tau0_8.csv'}


In [28]:
# Select strong inference submission (tau=0.5 primary; tau=0.7 as fallback if desired)
import shutil, os, pandas as pd
cand_primary = 'submission_cnn_strong_tau0_5.csv'
cand_secondary = 'submission_cnn_strong_tau0_7.csv'
choose = cand_primary if os.path.exists(cand_primary) else (cand_secondary if os.path.exists(cand_secondary) else None)
assert choose is not None, 'No strong inference submission found'
shutil.copyfile(choose, 'submission.csv')
print(f'Copied {choose} -> submission.csv')
print(pd.read_csv('submission.csv').head())

Copied submission_cnn_strong_tau0_5.csv -> submission.csv
   Id  Predicted
0   0      18454
1   1       5434
2   2      21267
3   3      10029
4   4      63426
